In [34]:
# Import packages
import pyspark
import dxpy
import dxdata
from pyspark.sql.functions import array_join,concat_ws,col


In [2]:
# Spark initialization (Done only once; do not rerun this cell unless you select Kernel -> Restart kernel).
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)

In [3]:
# Automatically discover dispensed database name and dataset id
dispensed_database = dxpy.find_one_data_object(
    classname='database', 
    name='app*', 
    folder='/', 
    name_mode='glob', 
    describe=True)
dispensed_database_name = dispensed_database['describe']['name']

dispensed_dataset = dxpy.find_one_data_object(
    typename='Dataset', 
    name='app*.dataset', 
    folder='/', 
    name_mode='glob')
dispensed_dataset_id = dispensed_dataset['id']

In [4]:
dataset = dxdata.load_dataset(id=dispensed_dataset_id)

In [5]:
participant = dataset['participant']

In [29]:
field_name_dict = {
    'sample_names': 'eid',
}

for idx in range(4):
    for arr in range(48):
        field_name_dict[f'medication{idx}_{arr}'] = f'p20003_i{idx}_a{arr}'


In [30]:
field_names = list(field_name_dict.values())

In [31]:
df = participant.retrieve_fields(names=field_names, engine=dxdata.connect(), coding_values="replace")

In [35]:
# Select all columns except 'eid' and concatenate them using '|'
df_transformed = df.withColumn("medications", concat_ws("|", *[col(c) for c in df.columns if c != "eid"]))

# Select only 'eid' and the new concatenated column
df_transformed = df_transformed.select("eid", "medications")


In [37]:
def upload_file_to_project(filename, proj_dir):
    dxpy.upload_local_file(filename, folder=proj_dir, parents=True)
    print(f"*********{filename} uploaded!!*********")
    return


In [38]:
pandas_df = df_transformed.toPandas()

In [39]:
proj_dir = f"/notebooks/bmi/data/"
filename = f"medications_raw.csv.gz"
pandas_df.to_csv(filename, index=False)
upload_file_to_project(filename, proj_dir)


*********medications_raw.csv.gz uploaded!!*********
